In [3]:
# Import statements
import pandas as pd

In [9]:
data = pd.read_csv('million-headlines/abcnews-date-text.csv', error_bad_lines=False, engine='python');
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text
print(len(documents))
print(documents[:5])

1103665
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [23]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ajay2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [26]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [27]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [28]:
dictionary = gensim.corpora.Dictionary(processed_docs)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [29]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [30]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (483, 1), (4014, 1)]

In [34]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
                                               bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 483 ("rain") appears 1 time.
Word 4014 ("dampen") appears 1 time.


In [35]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5892908644709983),
 (1, 0.38929657403503015),
 (2, 0.4964985198530063),
 (3, 0.5046520328695662)]


In [42]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2) # TAKES TIME

In [43]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.028*"elect" + 0.019*"say" + 0.018*"death" + 0.016*"hospit" + 0.015*"tasmanian" + 0.014*"labor" + 0.013*"deal" + 0.012*"china" + 0.011*"polit" + 0.011*"talk"
Topic: 1 
Words: 0.019*"nation" + 0.018*"coast" + 0.016*"help" + 0.016*"countri" + 0.015*"state" + 0.015*"health" + 0.014*"chang" + 0.013*"hour" + 0.013*"indigen" + 0.012*"water"
Topic: 2 
Words: 0.020*"canberra" + 0.018*"market" + 0.014*"rise" + 0.014*"west" + 0.014*"turnbul" + 0.014*"australian" + 0.013*"price" + 0.013*"share" + 0.011*"bank" + 0.010*"week"
Topic: 3 
Words: 0.063*"polic" + 0.023*"crash" + 0.019*"interview" + 0.018*"miss" + 0.018*"shoot" + 0.016*"arrest" + 0.015*"investig" + 0.013*"driver" + 0.011*"search" + 0.011*"offic"
Topic: 4 
Words: 0.029*"charg" + 0.028*"court" + 0.021*"murder" + 0.018*"woman" + 0.018*"face" + 0.016*"brisban" + 0.015*"live" + 0.015*"jail" + 0.014*"year" + 0.014*"perth"
Topic: 5 
Words: 0.035*"australia" + 0.022*"melbourn" + 0.021*"world" + 0.017*"open" + 0.014*"final" + 0.

In [44]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4) # TAKES TIME

In [45]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.014*"polic" + 0.013*"crash" + 0.012*"kill" + 0.010*"woman" + 0.009*"death" + 0.009*"shoot" + 0.008*"dead" + 0.008*"drum" + 0.008*"charg" + 0.008*"attack"
Topic: 1 Word: 0.012*"weather" + 0.008*"friday" + 0.006*"june" + 0.005*"bash" + 0.005*"syrian" + 0.005*"memori" + 0.005*"harvest" + 0.005*"billion" + 0.004*"mount" + 0.004*"run"
Topic: 2 Word: 0.010*"rural" + 0.010*"govern" + 0.007*"elect" + 0.006*"fund" + 0.006*"nation" + 0.006*"budget" + 0.006*"plan" + 0.005*"queensland" + 0.005*"labor" + 0.005*"council"
Topic: 3 Word: 0.024*"countri" + 0.022*"hour" + 0.011*"turnbul" + 0.008*"north" + 0.008*"sport" + 0.007*"korea" + 0.007*"david" + 0.006*"south" + 0.006*"august" + 0.006*"search"
Topic: 4 Word: 0.010*"grandstand" + 0.007*"tuesday" + 0.006*"dollar" + 0.005*"million" + 0.005*"blog" + 0.005*"say" + 0.005*"live" + 0.005*"station" + 0.005*"footag" + 0.005*"dump"
Topic: 5 Word: 0.019*"trump" + 0.010*"court" + 0.010*"podcast" + 0.009*"charg" + 0.009*"donald" + 0.008*"drug" 

In [51]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [49]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.4199989438056946	 
Topic: 0.026*"south" + 0.025*"kill" + 0.015*"island" + 0.013*"fall" + 0.011*"attack" + 0.010*"forc" + 0.009*"shark" + 0.009*"east" + 0.007*"northern" + 0.007*"bring"

Score: 0.2200000286102295	 
Topic: 0.019*"nation" + 0.018*"coast" + 0.016*"help" + 0.016*"countri" + 0.015*"state" + 0.015*"health" + 0.014*"chang" + 0.013*"hour" + 0.013*"indigen" + 0.012*"water"

Score: 0.2199999988079071	 
Topic: 0.035*"australia" + 0.022*"melbourn" + 0.021*"world" + 0.017*"open" + 0.014*"final" + 0.013*"donald" + 0.012*"sydney" + 0.010*"leagu" + 0.010*"take" + 0.010*"road"

Score: 0.02000104822218418	 
Topic: 0.019*"council" + 0.015*"power" + 0.013*"farmer" + 0.011*"guilti" + 0.010*"christma" + 0.010*"region" + 0.010*"feder" + 0.009*"program" + 0.009*"industri" + 0.009*"research"

Score: 0.019999999552965164	 
Topic: 0.028*"elect" + 0.019*"say" + 0.018*"death" + 0.016*"hospit" + 0.015*"tasmanian" + 0.014*"labor" + 0.013*"deal" + 0.012*"china" + 0.011*"polit" + 0.011*"talk"

In [50]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5827919244766235	 
Topic: 0.024*"countri" + 0.022*"hour" + 0.011*"turnbul" + 0.008*"north" + 0.008*"sport" + 0.007*"korea" + 0.007*"david" + 0.006*"south" + 0.006*"august" + 0.006*"search"

Score: 0.2571772336959839	 
Topic: 0.010*"final" + 0.009*"world" + 0.009*"leagu" + 0.008*"australia" + 0.006*"test" + 0.006*"beat" + 0.006*"open" + 0.005*"rugbi" + 0.005*"cricket" + 0.005*"win"

Score: 0.02000575140118599	 
Topic: 0.009*"farm" + 0.007*"marriag" + 0.006*"octob" + 0.006*"hobart" + 0.006*"peter" + 0.006*"thursday" + 0.005*"april" + 0.005*"wrap" + 0.005*"social" + 0.004*"wind"

Score: 0.020005209371447563	 
Topic: 0.014*"polic" + 0.013*"crash" + 0.012*"kill" + 0.010*"woman" + 0.009*"death" + 0.009*"shoot" + 0.008*"dead" + 0.008*"drum" + 0.008*"charg" + 0.008*"attack"

Score: 0.020004048943519592	 
Topic: 0.010*"rural" + 0.010*"govern" + 0.007*"elect" + 0.006*"fund" + 0.006*"nation" + 0.006*"budget" + 0.006*"plan" + 0.005*"queensland" + 0.005*"labor" + 0.005*"council"

Score: 0

In [53]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

In [54]:
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.28732940554618835	 Topic: 0.028*"elect" + 0.019*"say" + 0.018*"death" + 0.016*"hospit" + 0.015*"tasmanian"
Score: 0.2459985613822937	 Topic: 0.019*"nation" + 0.018*"coast" + 0.016*"help" + 0.016*"countri" + 0.015*"state"
Score: 0.18333333730697632	 Topic: 0.020*"canberra" + 0.018*"market" + 0.014*"rise" + 0.014*"west" + 0.014*"turnbul"
Score: 0.18333333730697632	 Topic: 0.063*"polic" + 0.023*"crash" + 0.019*"interview" + 0.018*"miss" + 0.018*"shoot"
Score: 0.01667201705276966	 Topic: 0.037*"trump" + 0.032*"australian" + 0.019*"queensland" + 0.014*"leav" + 0.014*"australia"
Score: 0.01666666567325592	 Topic: 0.029*"charg" + 0.028*"court" + 0.021*"murder" + 0.018*"woman" + 0.018*"face"
Score: 0.01666666567325592	 Topic: 0.035*"australia" + 0.022*"melbourn" + 0.021*"world" + 0.017*"open" + 0.014*"final"
Score: 0.01666666567325592	 Topic: 0.026*"south" + 0.025*"kill" + 0.015*"island" + 0.013*"fall" + 0.011*"attack"
Score: 0.01666666567325592	 Topic: 0.019*"council" + 0.015*"power"

In [55]:
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.6298670172691345	 Topic: 0.029*"charg" + 0.028*"court" + 0.021*"murder" + 0.018*"woman" + 0.018*"face"
Score: 0.23678265511989594	 Topic: 0.035*"australia" + 0.022*"melbourn" + 0.021*"world" + 0.017*"open" + 0.014*"final"
Score: 0.016674475744366646	 Topic: 0.063*"polic" + 0.023*"crash" + 0.019*"interview" + 0.018*"miss" + 0.018*"shoot"
Score: 0.016669554635882378	 Topic: 0.020*"canberra" + 0.018*"market" + 0.014*"rise" + 0.014*"west" + 0.014*"turnbul"
Score: 0.016669029369950294	 Topic: 0.036*"govern" + 0.022*"test" + 0.018*"rural" + 0.015*"break" + 0.012*"news"
Score: 0.016667934134602547	 Topic: 0.037*"trump" + 0.032*"australian" + 0.019*"queensland" + 0.014*"leav" + 0.014*"australia"
Score: 0.016667531803250313	 Topic: 0.019*"council" + 0.015*"power" + 0.013*"farmer" + 0.011*"guilti" + 0.010*"christma"
Score: 0.01666751690208912	 Topic: 0.026*"south" + 0.025*"kill" + 0.015*"island" + 0.013*"fall" + 0.011*"attack"
Score: 0.01666744239628315	 Topic: 0.019*"nation" + 0.018*"c